# Generative networks

RNN (Recurrent Neural Network)과 LSTM (Long Short Term Memory Cell) 및 GRU (Gated Recurrent Unit)와 같은 게이트 셀 변형모델은 언어 모델링(language modeling)을 위한 메커니즘을 제공했습니다. 즉, 단어 순서를 학습하고 시퀀스의 다음 단어에 대한 예측을 제공 할 수 있습니다. 이를 통해 일반 텍스트 생성, 기계 번역 및 이미지 캡션과 같은 **생성 작업(generative tasks)**에 RNN을 사용할 수 있습니다.

이전 단원에서 논의한 RNN 아키텍처에서 각 RNN 유닛은 다음 은닉 상태를 출력으로 생성했습니다. 그러나 각 반복 단위에 또 다른 출력을 추가하여 **sequence**(원래 시퀀스와 길이가 동일)를 출력할 수 있습니다. 또한 각 단계에서 입력을 받아들이지 않고 초기 상태 벡터를 취한 다음 일련의 출력을 생성하는 RNN 유닛을 사용할 수 있습니다.

이를 통해 아래 그림에 표시된 다양한 신경 아키텍처를 사용할 수 있습니다:

![Image showing common recurrent neural network patterns.](./images/unreasonable-effectiveness-of-rnn.jpg)
*Image from blog post [Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) by [Andrej Karpaty](http://karpathy.github.io/)*

* **One-to-one** 는 하나의 입력과 하나의 출력이있는 기성 신경망입니다.
* **One-to-many** 는 하나의 입력을 받아들이고 출력 시퀀스를 생성하는 생성 아키텍처(generative architecture)입니다. 예를 들어, 이미지에 대한 텍스트 설명을 생성하는 **이미지 캡션(image captioning)** 네트워크를 훈련 시키려면 이미지를 입력으로 사용하고 CNN을 통해 은닉 상태를 얻은 다음 반복 체인에서 캡션 단어를 생성하도록 할 수 있습니다.
* **Many-to-one** 는 텍스트 분류와 같이 이전 단원에서 설명한 RNN 아키텍처에 해당합니다.
* **Many-to-many** 또는 **sequence-to-sequence** 는 **기계 번역(machine translation)** 과 같은 작업에 해당합니다. 여기서 먼저 RNN이 입력 시퀀스의 모든 정보를 숨겨진 상태로 수집하고 다른 RNN 체인이 상태를 출력 시퀀스로 펼칩니다.

이 단원에서는 텍스트를 생성하는 데 도움이 되는 간단한 생성 모델에 중점을 둡니다. 간단하게 문자별로 텍스트를 생성하는 **문자 수준 네트워크(character-level network)** 를 구축해 보겠습니다. 훈련 중에는 텍스트 말뭉치를 가져와서 문자 시퀀스로 분할해야 합니다.

In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Building character vocabulary

문자 수준의 생성 네트워크를 구축하려면 텍스트를 단어 대신 개별 문자로 분할해야합니다. 이는 다른 tokenizer를 정의하여 수행할 수 있습니다:

In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.Vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.stoi['a']}")
print(f"Character with code 13 is {vocab.itos[13]}")

Vocabulary size = 84
Encoding of 'a' is 4
Character with code 13 is h


데이터 집합에서 텍스트를 인코딩하는 방법의 예를 살펴보자.

In [5]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([43,  4, 11, 11,  2, 26,  5, 23,  2, 38,  3,  4, 10,  9,  2, 31, 11,  4,
        21,  2, 38,  4, 14, 25,  2, 34,  8,  5,  6,  2,  5, 13,  3,  2, 38, 11,
         4, 14, 25,  2, 55, 37,  3, 15,  5,  3, 10,  9, 56,  2, 37,  3, 15,  5,
         3, 10,  9,  2, 29,  2, 26, 13,  6, 10,  5, 29,  9,  3, 11, 11,  3, 10,
         9, 27,  2, 43,  4, 11, 11,  2, 26,  5, 10,  3,  3,  5, 58,  9,  2, 12,
        21,  7,  8, 12, 11,  7,  8, 18, 61, 22,  4,  8, 12,  2,  6, 19,  2, 15,
        11,  5, 10,  4, 29, 14, 20,  8,  7, 14,  9, 27,  2,  4, 10,  3,  2,  9,
         3,  3,  7,  8, 18,  2, 18, 10,  3,  3,  8,  2,  4, 18,  4,  7,  8, 23])

## Training a generative RNN

텍스트를 생성하기 위해 RNN을 훈련시키는 방법은 다음과 같습니다. 각 단계에서 '`nchars` 길이의 문자 시퀀스를 가져 와서 각 입력 문자에 대해 다음 출력 문자를 생성하도록 네트워크에 요청합니다.

The way we will train RNN to generate text is the following. On each step, we will take a sequence of characters of length `nchars`, and ask the network to generate next output character for each input character:

![Image showing an example RNN generation of the word 'HELLO'.](./images/rnn-generate.png)

실제 시나리오에 따라 *end-of-sequence* `<eos>`와 같은 일부 특수 문자를 포함할 수도 있습니다. 우리의 경우 끝없는 텍스트 생성을 위해 네트워크를 훈련시키기를 원하므로 각 시퀀스의 크기를 `nchars` 토큰과 동일하게 수정합니다. 결과적으로 각 학습 예제는 `nchars` 길이의 입력과 `nchars` 길이의 출력으로 구성됩니다. 미니배치는 이러한 여러 시퀀스로 구성됩니다.

미니배치를 생성하는 방법은 길이가 `l`인 각 뉴스 텍스트를 가져 와서 가능한 모든 입출력 조합을 생성하는 것입니다 (이러한 조합이 `l-nchars`가 될 것입니다). 그들은 하나의 미니배치를 형성할 것이며 미니배치의 크기는 각 훈련 단계에서 다를 것입니다.

In [8]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[43,  4, 11,  ..., 18, 61, 22],
         [ 4, 11, 11,  ..., 61, 22,  4],
         [11, 11,  2,  ..., 22,  4,  8],
         ...,
         [37,  3, 15,  ...,  4, 18,  4],
         [ 3, 15,  5,  ..., 18,  4,  7],
         [15,  5,  3,  ...,  4,  7,  8]], device='cuda:0'),
 tensor([[ 4, 11, 11,  ..., 61, 22,  4],
         [11, 11,  2,  ..., 22,  4,  8],
         [11,  2, 26,  ...,  4,  8, 12],
         ...,
         [ 3, 15,  5,  ..., 18,  4,  7],
         [15,  5,  3,  ...,  4,  7,  8],
         [ 5,  3, 10,  ...,  7,  8, 23]], device='cuda:0'))

이제 생성 네트워크를 정의하겠습니다. 이전 단원에서 논의한 모든 순환 셀(단순 RNN, LSTM 또는 GRU)을 기반으로 할 수 있습니다. 이 예시에서는 LSTM을 사용합니다.

네트워크는 문자를 입력으로 사용하고 사전(vocabulary) 크기가 매우 작기 때문에 임베딩 레이어가 필요하지 않으며 원-핫 인코딩된 입력은 LSTM 셀로 직접 이동할 수 있습니다. 그러나 문자 번호(character numbers)를 입력으로 전달하기 때문에 LSTM으로 전달하기 전에 원-핫 인코딩해야 합니다. 이는 `forward` 패스 중에 `one_hot` 함수를 호출하여 수행됩니다. 출력 인코더는 숨겨진 상태를 원-핫 인코딩된 출력으로 변환하는 선형 레이어입니다.

In [9]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

우리는 훈련 중에 생성된 텍스트를 샘플링할 수 있기를 원합니다. 이를 위해 초기 문자열 `start`에서 시작하여 길이가 `size`인 출력 문자열을 생성하는 `generate`함수를 정의합니다.

작동 방식은 다음과 같습니다. 먼저 전체 시작 문자열을 네트워크를 통해 전달하고 출력 상태 `s`와 다음 예측 문자 `out`을 가져옵니다. `out`은 원-핫 인코딩되므로 `argmax`를 사용하여 사전에서 `nc` 문자의 인덱스를 가져오고 `itos`를 사용하여 실제 문자를 알아내고 결과 `chars` 문자에 추가합니다. 하나의 문자를 생성하는이 프로세스는 필요한 수의 문자를 생성하기 위해 `size`번 반복됩니다.

In [13]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.itos[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

이제 훈련을 합시다! 훈련 루프는 이전의 모든 예제에서와 거의 동일하지만 정확도 대신 1000 epoch마다 샘플로 생성된 텍스트를 출력합니다.

손실을 계산하는 방식에 특별한 주의가 필요합니다. 원-핫 인코딩된 출력 `out`과 예상되는 텍스트 `text_out`(문자 인덱스 목록)이 주어지면 손실을 계산해야 합니다. 다행히도 `cross_entropy` 함수는 정규화되지 않은 네트워크 출력을 첫 번째 인수로 예상하고 클래스 번호를 두 번째 인수로 예상합니다. 또한 미니배치 크기에 대한 자동 평균을 수행합니다.

또한 너무 오래 기다리지 않도록 `samples_to_train` 샘플로 훈련을 제한합니다. 여러 epoch(이 경우이 코드 주위에 또 다른 루프를 생성해야 함)에 대해 더 긴 훈련을 시도하고 시도하는 것이 좋습니다.

In [14]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.442246913909912
today ggrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrgrg
Current loss = 2.1178359985351562
today and a could a the to the to the to the to the to the to the to the to the to the to the to the to th
Current loss = 1.6465336084365845
today on Tuesday the company to the United States and a policing to the United States and a policing to th
Current loss = 2.3716814517974854
today to the United States and a new men to the United States and a new men to the United States and a new
Current loss = 1.6844098567962646
today of the first the first the first the first the first the first the first the first the first the fir
Current loss = 1.702707052230835
today of the United States a said the United States a said the United States a said the United States a sa
Current loss = 1.9633255004882812
today of the first the first the first the first the first the first the first the first the first the fir
Current loss = 

이 예제는 이미 꽤 좋은 텍스트를 생성하지만 몇 가지 방법으로 더 개선 할 수 있습니다.

This example already generates some pretty good text, but it can be further improved in several ways:
* **Better minibatch generation**. 우리가 훈련 데이터를 준비하는 방법은 하나의 샘플에서 하나의 미니배치를 생성하는 것이 었습니다. 미니배치는 모두 크기가 다르고 텍스트가 `nchars`보다 길이가 작기 때문에 일부는 생성할 수 없어서 이상적이지 않습니다. 또한 작은 미니배치는 GPU를 충분히 로드하지 못합니다. 모든 샘플에서 하나의 큰 텍스트 청크(chunk)를 얻은 다음 모든 입출력 쌍을 생성하고 섞고 동일한 크기의 미니배치를 생성하는 것이 현명할 것입니다.
* **Multilayer LSTM**. 모델 구축시 LSTM 셀의 2개 또는 3개의 층을 시도하는 것이 좋습니다. 이전 단원에서 언급했듯이 LSTM의 각 레이어는 텍스트에서 특정 패턴을 추출하며, 문자 수준 생성기의 경우 단어와 단어 조합에 대해 더 낮은 LSTM 수준이 음절 추출을 담당하고 더 높은 수준을 기대할 수 있다. 이는 레이어 수 파라미터를 LSTM 생성자에 전달하여 간단히 구현할 수 있습니다.
* **GRU units** 으로 실험하여 어떤 것이 더 나은 성능을 내고 **다른 은닉층 크기** 를 사용하는지 확인할 수도 있습니다. 은닉 층 크기가 너무 크면 과적합을 초래할 수 있고, 크기가 작을수록 좋은 결과가 생성되지 않을 수 있습니다.

## Soft text generation and temperature

`generate`의 이전 정의에서 우리는 항상 생성된 텍스트의 다음 문자로 예측 확률이 가장 높은 문자를 선택했습니다. 이로 인해 다음 예제처럼 텍스트가 동일한 문자 시퀀스 사이에서 반복해서 "순환(cycled)"되는 경우가 많습니다.

```
today of the second the company and a second the company ...
```

그러나 다음 문자에 대한 확률 분포를 살펴보면 몇 가지 가장 높은 확률 간의 차이가 크지 않을 수 있습니다. 한 문자의 확률 0.2, 다른 문자 0.19로 나타날 수 있습니다. 예를 들어 시퀀스 '*play*'에서 다음 문자를 찾을 때 다음 문자는 공백 또는 **e** (예 : *player*).

두 번째로 확률이 높은 문자를 선택하는 것이 여전히 의미있는 텍스트로 이어질 수 있기 때문에 제일 높은 확률값의 문자를 선택하는 것이 항상 "공정(fair)"한 것은 아니라는 결론에 도달합니다. 네트워크 출력에 의해 주어진 확률 분포에서 문자를 **샘플(sample)** 하는 것이 더 현명합니다.

이 샘플링은 소위 **multinomial distribution**를 구현하는 `multinomial` 함수를 사용하여 수행할 수 있습니다. 이 **soft** 텍스트 생성을 구현하는 함수는 다음과 같이 정의됩니다.

In [15]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.itos[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and to has a software to in the first the power the gold medal was of the first and succer to the company will a report the first the and the gain the company in the and a new a report a pack of the four the first the company of the such with the half to a security to the and a success the first she

--- Temperature = 0.8
Today drud out of the three-rent possiem that sales purssion has finminiaty women's from NAC Inc. (AP) -- Shimbon has weel with a may stelight first three flaw gold from their a scent, big study with a nighting sovicturner has slarh football at a hour of Angelage discression, into cubs, US year player sor

--- Temperature = 1.0
Today by compoy, said to hup the couns ay rrope iist\fill sinie-5-1- than he of a fightier Corp. the Vew, Mkli Unite Hold Austria on Tuesday resfare rextarted in the new has buy thisnillials thrust first capuration of the it larget expected the ir edulagy Airin Penny after Emonet Cuc Washieve an are Gurry

--- Temper

우리는 가장 높은 확률을 고수해야 하는 정도를 나타내는 데 사용되는 **temperature**라는 매개 변수를 하나 더 도입했습니다. temperature가 1.0이면 공정한 다항(multinomial) 샘플링을 수행하고, temperature가 무한대에 도달하면 모든 확률이 같게되고 다음 문자를 무작위로 선택합니다. 아래 예에서 temperature를 너무 많이 올리면 텍스트가 무의미해지고 0에 가까워지면 "순환된(cycled)" 텍스트와 유사함을 알 수 있습니다.